In [18]:
from datetime import datetime
import requests
import json
import pandas as pd

# APi MercadoLibre

- Para trabajar con la API debemos crearnos antes que nada una aplicacion en devs.mercadolibre
- Luego generar las credenciales necesarias
- Por ultimo hacer los http requests necesarios para generar el TOKEN de autorizacion (via postman, luego via requests)

*Es importante tener dentro del scope de la aplicacion chequeada la opcion de "offline access" para poder utilizar el refresh token via offline.

CREDENCIALES DE LA APLICACION

In [19]:
#Credentials in the App
clientID= "2476757368160136"
url_localHost= "https://localhost:3000"
client_secret = "1PhHHlErkfFQgSAHiYbc3RsZarUGaWIs"
code="TG-63f413cf987037000176d55a-50585801"

url = "https://auth.mercadolibre.com.ar/authorization?response_type=code&client_id={clientID}}&redirect_uri={url_localHost}"
response = requests.get(url)


Obtencion del Access Token y Refresh Token via Postman

In [20]:
#Obtencion de Access Token y Refresh Token con Postman

{
    "access_token": "APP_USR-2476757368160136-022020-a0cfdab2b77c8264b6d528c62ecabdc4-50585801",
    "token_type": "Bearer",
    "expires_in": 21600,
    "scope": "offline_access read",
    "user_id": 50585801,
    "refresh_token": "TG-63f4153f0bdebf0001594c8f-50585801"
}

{'access_token': 'APP_USR-2476757368160136-022020-a0cfdab2b77c8264b6d528c62ecabdc4-50585801',
 'token_type': 'Bearer',
 'expires_in': 21600,
 'scope': 'offline_access read',
 'user_id': 50585801,
 'refresh_token': 'TG-63f4153f0bdebf0001594c8f-50585801'}

Utilizando la herramienta de traduccion de codigo de Postman generamos el Python Requests para obtener el access_token y refresh_token


In [21]:
#Obtencion codigo Python (Requests) a partir de Postman

url = "https://api.mercadolibre.com/oauth/token"

payload='grant_type=authorization_code&client_id=2476757368160136&client_secret=1PhHHlErkfFQgSAHiYbc3RsZarUGaWIs&code=TG-63f41aba607d7b0001d6de3f-50585801&redirect_uri=https%3A%2F%2Flocalhost%3A3000'
headers = {
  'accept': 'application/json',
  'content-type': 'application/x-www-form-urlencoded'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"message":"Error validating grant. Your authorization code or refresh token may be expired or it was already used","error":"invalid_grant","status":400,"cause":[]}


Actualizamos el Access Token utilizando Refresh Token con Pyhton

In [22]:
#Actualizando Access Token usando Refresh Token con Pyhton

import requests

url = "https://api.mercadolibre.com/oauth/token"

payload='grant_type=refresh_token&client_id=2476757368160136&client_secret=1PhHHlErkfFQgSAHiYbc3RsZarUGaWIs&refresh_token=TG-63f41af3987037000176eb90-50585801'
headers = {
  'accept': 'application/json',
  'content-type': 'application/x-www-form-urlencoded'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"access_token":"APP_USR-2476757368160136-022117-b1cb5b7fbbdf2728659ef8d4193a2525-50585801","token_type":"Bearer","expires_in":21600,"scope":"offline_access read","user_id":50585801,"refresh_token":"TG-63f41af3987037000176eb90-50585801"}


In [23]:
print("Status Code:", response.status_code)

Status Code: 200


Consultamos las categorias disponibles dentro del sitio de Mercado Libre Argentina (*MLA)

In [24]:
#https://api.mercadolibre.com/sites/MLA/categories

url = "https://api.mercadolibre.com/sites/MLA/categories"
response = requests.get(url)


In [25]:
response.url

'https://api.mercadolibre.com/sites/MLA/categories'

Probamos uno de los endpoints en la APi de MELI, Lista de Categorias disponibles en el sitio

Lista de las categorias en MELI Argentina

In [26]:
#Lista de todas las categorias en MELi
response.text

'[{"id":"MLA5725","name":"Accesorios para Vehículos"},{"id":"MLA1512","name":"Agro"},{"id":"MLA1403","name":"Alimentos y Bebidas"},{"id":"MLA1071","name":"Animales y Mascotas"},{"id":"MLA1367","name":"Antigüedades y Colecciones"},{"id":"MLA1368","name":"Arte, Librería y Mercería"},{"id":"MLA1743","name":"Autos, Motos y Otros"},{"id":"MLA1384","name":"Bebés"},{"id":"MLA1246","name":"Belleza y Cuidado Personal"},{"id":"MLA1039","name":"Cámaras y Accesorios"},{"id":"MLA1051","name":"Celulares y Teléfonos"},{"id":"MLA1648","name":"Computación"},{"id":"MLA1144","name":"Consolas y Videojuegos"},{"id":"MLA1500","name":"Construcción"},{"id":"MLA1276","name":"Deportes y Fitness"},{"id":"MLA5726","name":"Electrodomésticos y Aires Ac."},{"id":"MLA1000","name":"Electrónica, Audio y Video"},{"id":"MLA2547","name":"Entradas para Eventos"},{"id":"MLA407134","name":"Herramientas"},{"id":"MLA1574","name":"Hogar, Muebles y Jardín"},{"id":"MLA1499","name":"Industrias y Oficinas"},{"id":"MLA1459","name":"

In [27]:
#/categories/$CATEGORY_ID/attributes
#Categoria AGRO

url = "https://api.mercadolibre.com/categories/MLA1512/attributes"
response = requests.get(url)


Con el endpoint ""attributes"" podemos averiguar los atributos correspondientes a una categoria en particular que querramos

In [28]:
#/sites/$SITE_ID/domain_discovery/search?q=$Q

##Predictor de categorías. Devuelve la categoría correspondente para enumerar un artículo basándose en el título, dominio e/ou atributos	

Con el endpoint ""Category ID" podemos ver una lista de items dentro de una categoria de productos para el sitio de MLA

In [29]:
#/sites/$SITE_ID/search?category=$CATEGORY_ID

url = "https://api.mercadolibre.com//sites/MLA/search?category=MLA1648"
response = requests.get(url)



En el atributo `response.text` de la respuesta, obtenemos el contenido del mensaje. Por ser esta una request tipo get, pueden hacer la prueba de ingresar a la URL de la request en su navegador, y observarán el json del resultado que aquí obtenemos en forma programática.

El texto viene como un string, pero queremos poder tratarlo como una estructura de datos, con lo cual vamos a parsear el json con la librería `json` de Python. Esta librería viene con cualquier instalación de Python, por lo cual no requiere setup adicional. `json.loads()` convertirá la respuesta en objetos nativos de Python (diccionarios y listas), con el fin de poder armar nuestro dataset.

En el caso particular de la respuesta de Mercadolibre, dentro del json encontramos una clave "results", en la cual encontraremos toda publicación perteneciente a la respuesta. Vamos a quedarnos con los resultados en nuestra variable `results`.

In [30]:
data = json.loads(response.text)
results=data["results"]

<a id="section_df"></a> 
###  2- Formando un DataFrame

Vamos a crear un DataFrame de publicaciones de Mercadolibre con el fin de predecir la categoria a la que pertenece un producto. Para ello, queremos formar un DataFrame de pandas y formar un CSV. Usar el constructor de DataFrame sobre nuestra variable `results` tiene varias limitaciones vamos a ver algunas de ellas a continuación.

In [31]:
df=pd.DataFrame(results)
df.head()

,id,title,condition,thumbnail_id,catalog_product_id,listing_type_id,permalink,buying_mode,site_id,category_id,...,attributes,installments,winner_item_id,catalog_listing,discounts,promotions,inventory_id,differential_pricing,variation_filters,variations_data
0,MLA1338040424,Adaptador Usb C Compatible Macbook Pro Air Hdm...,new,854371-MLA53090105197_122022,MLA20995410,gold_special,https://www.mercadolibre.com.ar/adaptador-usb-...,buy_it_now,MLA,MLA9930,...,"[{'id': 'BRAND', 'name': 'Marca del cable', 'v...","{'quantity': 6, 'amount': 2828.32, 'rate': 48....",None,True,None,[],GTQV39618,NaN,NaN,NaN
1,MLA1163263733,"Notebook Dell Inspiron 3515 Plateada 15.5 , Am...",new,805951-MLA50865156506_072022,MLA19034498,gold_special,https://www.mercadolibre.com.ar/notebook-dell-...,buy_it_now,MLA,MLA1652,...,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...","{'quantity': 6, 'amount': 51922.25, 'rate': 48...",None,True,None,[],None,NaN,NaN,NaN
2,MLA932293921,Pc Armada Gamer Amd Ryzen 5 5600g 6/12 Nucleos...,new,884368-MLA53087339003_122022,None,gold_special,https://articulo.mercadolibre.com.ar/MLA-93229...,buy_it_now,MLA,MLA1649,...,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...","{'quantity': 6, 'amount': 42774, 'rate': 48.35...",None,NaN,None,[],None,NaN,NaN,NaN
3,MLA1103568884,"Apple Macbook Air (13 Pulgadas, 2020, Chip M1,...",new,801112-MLA46516512347_062021,MLA17828518,gold_special,https://www.mercadolibre.com.ar/apple-macbook-...,buy_it_now,MLA,MLA1652,...,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...","{'quantity': 6, 'amount': 88787.23, 'rate': 48...",None,True,None,[],None,NaN,NaN,NaN
4,MLA874819289,Pc Armada Intel Core I5 8gb Ssd240 W10 Office,new,691246-MLA49643833761_042022,None,gold_special,https://articulo.mercadolibre.com.ar/MLA-87481...,buy_it_now,MLA,MLA1649,...,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...","{'quantity': 6, 'amount': 21634.13, 'rate': 48...",None,NaN,None,[],RYIP70483,NaN,NaN,NaN


En este primer resultado podemos observar varios de los problemas de los que hablábamos anteriormente:
* Muchas de nuestras columnas son diccionarios. ¿Cómo podemos acceder a los campos en cada uno?
* Sólo tenemos 50 registros. ¿Qué pasa si modificamos el parámetro `limit` en nuestra request?
* Muchas de las publicaciones que vemos no refieren al producto que buscamos, sino que nos hablan de accesorios para el mismo. ¿Es posible filtrar estas respuestas?

<a id="section_json_normalize"></a> 
#### 2.1- Parseando un JSON con Pandas

Afortunadamente, como dijimos antes, el formato JSON se ha vuelto un estándar ampliamente difundido en los respuestas de APIs y, como tal, Pandas tiene su propia implementación para leer información de JSONs y expandir los subcampos.

Por todo lo susodicho, la función `json_normalize` de Pandas resuelve en forma sencilla y completa el primero de los tres problemas que teníamos anteriormente. Veamos cómo queda ese DataFrame:

In [32]:
df_expand=pd.json_normalize(results)
df_expand.head()

,id,title,condition,thumbnail_id,catalog_product_id,listing_type_id,permalink,buying_mode,site_id,category_id,...,variations_data.176349803201.thumbnail,variations_data.176349803201.ratio,variations_data.176349803201.name,variations_data.176349803201.pictures_qty,variations_data.176349803201.inventory_id,variations_data.174201156322.thumbnail,variations_data.174201156322.ratio,variations_data.174201156322.name,variations_data.174201156322.pictures_qty,variations_data.174201156322.inventory_id
0,MLA1338040424,Adaptador Usb C Compatible Macbook Pro Air Hdm...,new,854371-MLA53090105197_122022,MLA20995410,gold_special,https://www.mercadolibre.com.ar/adaptador-usb-...,buy_it_now,MLA,MLA9930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MLA1163263733,"Notebook Dell Inspiron 3515 Plateada 15.5 , Am...",new,805951-MLA50865156506_072022,MLA19034498,gold_special,https://www.mercadolibre.com.ar/notebook-dell-...,buy_it_now,MLA,MLA1652,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MLA932293921,Pc Armada Gamer Amd Ryzen 5 5600g 6/12 Nucleos...,new,884368-MLA53087339003_122022,None,gold_special,https://articulo.mercadolibre.com.ar/MLA-93229...,buy_it_now,MLA,MLA1649,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MLA1103568884,"Apple Macbook Air (13 Pulgadas, 2020, Chip M1,...",new,801112-MLA46516512347_062021,MLA17828518,gold_special,https://www.mercadolibre.com.ar/apple-macbook-...,buy_it_now,MLA,MLA1652,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MLA874819289,Pc Armada Intel Core I5 8gb Ssd240 W10 Office,new,691246-MLA49643833761_042022,None,gold_special,https://articulo.mercadolibre.com.ar/MLA-87481...,buy_it_now,MLA,MLA1649,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Podemos ver que nuestro nuevo DataFrame tiene muchas más columnas. Comparemos ambos índices:

In [33]:
print("DataFrame original:", list(df.columns))
print()
print("DataFrame expandido:", list(df_expand.columns))
print()

DataFrame original: ['id', 'title', 'condition', 'thumbnail_id', 'catalog_product_id', 'listing_type_id', 'permalink', 'buying_mode', 'site_id', 'category_id', 'domain_id', 'thumbnail', 'currency_id', 'order_backend', 'price', 'original_price', 'sale_price', 'sold_quantity', 'available_quantity', 'official_store_id', 'use_thumbnail_id', 'accepts_mercadopago', 'tags', 'shipping', 'stop_time', 'seller', 'seller_address', 'address', 'attributes', 'installments', 'winner_item_id', 'catalog_listing', 'discounts', 'promotions', 'inventory_id', 'differential_pricing', 'variation_filters', 'variations_data']

DataFrame expandido: ['id', 'title', 'condition', 'thumbnail_id', 'catalog_product_id', 'listing_type_id', 'permalink', 'buying_mode', 'site_id', 'category_id', 'domain_id', 'thumbnail', 'currency_id', 'order_backend', 'price', 'original_price', 'sale_price', 'sold_quantity', 'available_quantity', 'official_store_id', 'use_thumbnail_id', 'accepts_mercadopago', 'tags', 'stop_time', 'attrib

Sin embargo, el DataFrame expandido trae dos problemas adicionales:
1. Genera demasiadas columnas, ya que no todos los subcampos nos son relevantes.
1. Muestra los niveles de expansión por medio de puntos, lo cual nos molestaría para la notación de atributos (por ejemplo `df_expand.seller.seller_reputation.real_level` nos arrojaría un error, en lugar de devolver la serie `df_expand["seller.seller_reputation.real_level"]`.

Estos dos problemas se resuelven sencillamente, renombrando las columnas y luego quedándonos sólo con las relevantes. Ensallaremos una solución para estos problemas, que luego reproduciremos en nuestro dataset final.

<a id="section_data_wrangling"></a> 
#### 2.2- Data Wrangling

Antes de generar un DataFrame de un tamaño importante para efectuar predicciones, queremos dejar armadas funciones para poder limpiarlo en forma rápida. Comenzaremos renombrando columnas para remover los puntos.

In [34]:
def rename_json_cols(df):
    df = df.copy()
    df.columns = [col.replace(".", "__") for col in df.columns]
    return df

In [35]:
df_no_dots = rename_json_cols(df_expand)
df_no_dots.head()

,id,title,condition,thumbnail_id,catalog_product_id,listing_type_id,permalink,buying_mode,site_id,category_id,...,variations_data__176349803201__thumbnail,variations_data__176349803201__ratio,variations_data__176349803201__name,variations_data__176349803201__pictures_qty,variations_data__176349803201__inventory_id,variations_data__174201156322__thumbnail,variations_data__174201156322__ratio,variations_data__174201156322__name,variations_data__174201156322__pictures_qty,variations_data__174201156322__inventory_id
0,MLA1338040424,Adaptador Usb C Compatible Macbook Pro Air Hdm...,new,854371-MLA53090105197_122022,MLA20995410,gold_special,https://www.mercadolibre.com.ar/adaptador-usb-...,buy_it_now,MLA,MLA9930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MLA1163263733,"Notebook Dell Inspiron 3515 Plateada 15.5 , Am...",new,805951-MLA50865156506_072022,MLA19034498,gold_special,https://www.mercadolibre.com.ar/notebook-dell-...,buy_it_now,MLA,MLA1652,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MLA932293921,Pc Armada Gamer Amd Ryzen 5 5600g 6/12 Nucleos...,new,884368-MLA53087339003_122022,None,gold_special,https://articulo.mercadolibre.com.ar/MLA-93229...,buy_it_now,MLA,MLA1649,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MLA1103568884,"Apple Macbook Air (13 Pulgadas, 2020, Chip M1,...",new,801112-MLA46516512347_062021,MLA17828518,gold_special,https://www.mercadolibre.com.ar/apple-macbook-...,buy_it_now,MLA,MLA1652,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MLA874819289,Pc Armada Intel Core I5 8gb Ssd240 W10 Office,new,691246-MLA49643833761_042022,None,gold_special,https://articulo.mercadolibre.com.ar/MLA-87481...,buy_it_now,MLA,MLA1649,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


En primer lugar removeremos las columnas geográficas, con todos los subelementos correspondientes, así como los ids innecesarios, y las variables categóricas que sólo tengan un valor posible.

In [36]:
def drop_columns(df, drop_geo=True, drop_ids=True, drop_uniques=True):
    """
    Función para eliminar columnas innecesarias del DataFrame.
    
    Parameters
    ----------
    
    drop_geo : bool
        Booleano para definir si eliminamos las veriables geográficas. Default, True.
    drop_ids: bool
        Booleano para definir si eliminamos las veriables de IDs y similares. Default, True.
    drop_uniques: bool
        Booleano para definir si eliminamos las veriables con un solo valor posible. Default, True.

    """
    df = df.copy()
    # Variables geográficas:
    if drop_geo:
        nogeo_mask = df.columns[
            ~df.columns.str.contains(r"^address__|^seller_address__|^seller__eshop")
        ]
        df = df[nogeo_mask]
    if drop_uniques:
        series_uniques = df.apply(lambda x: len(x.value_counts()) != 1).values
        df = df.loc[:, series_uniques]
    if drop_ids:
        for col in ["seller__permalink", "catalog_product_id","thumbnail", "attributes", "original_price", "official_store_id"]:
            try:
                df = df.drop(col, axis=1)
            except Exception:
                print(f"{col} ya fue eliminada en chequeos anteriores.")
        series_ids = df.apply(lambda x: len(x.value_counts()) != len(df)).values
        df = df.loc[:, series_ids]
    # También columnas excesivamente nulas.
    series_nulls = df.isnull().mean() < 0.6
    df = df.loc[:, series_nulls]

    
    return df

In [37]:
df_pruned = drop_columns(df_no_dots)

Asi queda el dataFrame con las columnas removidas y limpias.

In [38]:
df_pruned.head()


,thumbnail_id,listing_type_id,category_id,domain_id,price,sold_quantity,available_quantity,tags,stop_time,inventory_id,...,seller__seller_reputation__transactions__total,seller__seller_reputation__metrics__sales__completed,seller__seller_reputation__metrics__claims__rate,seller__seller_reputation__metrics__claims__value,seller__seller_reputation__metrics__delayed_handling_time__rate,seller__seller_reputation__metrics__delayed_handling_time__value,seller__seller_reputation__metrics__cancellations__rate,seller__seller_reputation__metrics__cancellations__value,installments__amount,installments__rate
0,854371-MLA53090105197_122022,gold_special,MLA9930,MLA-DATA_CABLES_AND_ADAPTERS,11439.12,5,1,"[brand_verified, good_quality_thumbnail, light...",2043-02-01T04:00:00.000Z,GTQV39618,...,18515,2208,0.0135,35,0.0172,40,0.0190,49,2828.32,48.35
1,805951-MLA50865156506_072022,gold_special,MLA1652,MLA-NOTEBOOKS,209999.00,5,5000,"[extended_warranty_eligible, good_quality_pict...",2042-08-31T04:00:00.000Z,None,...,53867,1878,0.0075,15,0.0010,2,0.0065,13,51922.25,48.35
2,884368-MLA53087339003_122022,gold_special,MLA1649,MLA-DESKTOP_COMPUTERS,172999.00,500,500,"[extended_warranty_eligible, good_quality_pict...",2041-07-27T04:00:00.000Z,None,...,29741,3694,0.0060,24,0.0068,23,0.0035,14,42774.00,48.35
3,801112-MLA46516512347_062021,gold_special,MLA1652,MLA-NOTEBOOKS,359099.00,25,1,"[good_quality_picture, good_quality_thumbnail,...",2041-09-11T04:00:00.000Z,None,...,7845,990,0.0075,8,0.0000,0,0.0000,0,88787.23,48.35
4,691246-MLA49643833761_042022,gold_special,MLA1649,MLA-DESKTOP_COMPUTERS,87499.00,500,1,"[loyalty_discount_eligible, moderation_penalty...",2040-08-18T04:00:00.000Z,RYIP70483,...,7047,1135,0.0097,12,0.0242,28,0.0040,5,21634.13,48.35


In [39]:
df_pruned.columns

Index(['thumbnail_id', 'listing_type_id', 'category_id', 'domain_id', 'price',
       'sold_quantity', 'available_quantity', 'tags', 'stop_time',
       'inventory_id', 'shipping__logistic_type', 'shipping__free_shipping',
       'shipping__tags', 'seller__id', 'seller__nickname',
       'seller__registration_date', 'seller__tags',
       'seller__seller_reputation__power_seller_status',
       'seller__seller_reputation__transactions__canceled',
       'seller__seller_reputation__transactions__completed',
       'seller__seller_reputation__transactions__ratings__negative',
       'seller__seller_reputation__transactions__ratings__neutral',
       'seller__seller_reputation__transactions__ratings__positive',
       'seller__seller_reputation__transactions__total',
       'seller__seller_reputation__metrics__sales__completed',
       'seller__seller_reputation__metrics__claims__rate',
       'seller__seller_reputation__metrics__claims__value',
       'seller__seller_reputation__metrics_

Con esto nos quedamos con un df bastante terminado, pero tenemos dos columnas que nos resultan interesantes por ser fechas: `seller__registration_date` y `stop_time`. Queremos convertir la primera en años de actividad del vendedor, y la segunda en días restantes a la publicación.

In [40]:
def parse_dates(df):
    """Convierte 'stop_time' en 'days_remaining' y 'seller__registration_date' en 'years_active'."""
    df = df.copy()
    df["days_remaining"] = (
        pd.to_datetime(df['stop_time'], utc=True) - pd.Timestamp(datetime.utcnow(), tz="UTC")
    ).dt.days
    df["years_active"] = (
        pd.Timestamp(datetime.utcnow(), tz="UTC") -  pd.to_datetime(df['seller__registration_date'], utc=True)
    ).dt.days // 365
    df = df.drop(["stop_time", "seller__registration_date"], axis=1)
    return df

In [41]:
df_parsed = parse_dates(df_pruned)
df_parsed.head()

,thumbnail_id,listing_type_id,category_id,domain_id,price,sold_quantity,available_quantity,tags,inventory_id,shipping__logistic_type,...,seller__seller_reputation__metrics__claims__rate,seller__seller_reputation__metrics__claims__value,seller__seller_reputation__metrics__delayed_handling_time__rate,seller__seller_reputation__metrics__delayed_handling_time__value,seller__seller_reputation__metrics__cancellations__rate,seller__seller_reputation__metrics__cancellations__value,installments__amount,installments__rate,days_remaining,years_active
0,854371-MLA53090105197_122022,gold_special,MLA9930,MLA-DATA_CABLES_AND_ADAPTERS,11439.12,5,1,"[brand_verified, good_quality_thumbnail, light...",GTQV39618,fulfillment,...,0.0135,35,0.0172,40,0.0190,49,2828.32,48.35,7284,13
1,805951-MLA50865156506_072022,gold_special,MLA1652,MLA-NOTEBOOKS,209999.00,5,5000,"[extended_warranty_eligible, good_quality_pict...",None,cross_docking,...,0.0075,15,0.0010,2,0.0065,13,51922.25,48.35,7130,12
2,884368-MLA53087339003_122022,gold_special,MLA1649,MLA-DESKTOP_COMPUTERS,172999.00,500,500,"[extended_warranty_eligible, good_quality_pict...",None,cross_docking,...,0.0060,24,0.0068,23,0.0035,14,42774.00,48.35,6730,16
3,801112-MLA46516512347_062021,gold_special,MLA1652,MLA-NOTEBOOKS,359099.00,25,1,"[good_quality_picture, good_quality_thumbnail,...",None,cross_docking,...,0.0075,8,0.0000,0,0.0000,0,88787.23,48.35,6776,14
4,691246-MLA49643833761_042022,gold_special,MLA1649,MLA-DESKTOP_COMPUTERS,87499.00,500,1,"[loyalty_discount_eligible, moderation_penalty...",RYIP70483,cross_docking,...,0.0097,12,0.0242,28,0.0040,5,21634.13,48.35,6387,10


Aqui modificamos la funcion ""full_request"" para poder traernos productos con el endpoint para items listados por categorias. 

**NOTA IMPORTANTE:** la API tiene un límite de requests máximas permitidas, así que no ejecuten esta celda muchas veces seguidas.

In [49]:
#Funcion para buscar por Categoria
#Modificamos levemente la funcion de busqueda "Search" para poder buscar por categoria

#RECURSO
#/sites/$SITE_ID/search?category=$CATEGORY_ID

#DESCRIPCION
#Obtener ítems listados en una categoría.


def full_request(category=None, limit=50, total_obs=20000, save=True, access_token=None):
    url = "https://api.mercadolibre.com/sites/MLA/search?"
    results = []
    params = {
        "category": str,
        "limit": 50,
        "condition": "new"
    }
    #if category is None:
    #    params = {
    #        "category": "value",
    #        "limit": 50,
    #        "condition": "new",
    #}
    #    response_ = requests.get(url, params=params)
    #    cats_ = pd.DataFrame(json.loads(response_.text)["results"]).category_id.value_counts()
    #    category = cats_.index[0]
    #    del cats_, response_
    params["category"] = category
    params["limit"] = limit
    params["access_token"] = access_token
    for offset in range(0, total_obs, limit):  # Paginamos usando el parámetro
        try:
            params["offset"] = offset
            response = requests.get(url, params=params)
            results += response.json()["results"]
        except KeyError:
            print(
                "Error: El máximo de datos permitidos fue superado o no puede encontrarse más publicaciones.",
                f"Se obtuvieron {len(results)} publicaciones."
            )
            break
    df = pd.json_normalize(results)
    df = rename_json_cols(df)
    df = drop_columns(df)
    df = parse_dates(df)
    if save:
        df.to_csv(f'./Data/{category.lower().replace(" ", "_")}_meli.csv', index=False)
    return df

In [63]:
final_df = full_request(category="MLA1459", limit=50,total_obs=20000, access_token="APP_USR-2476757368160136-022117-441c5f21d80a803f64fb0b416c5b9c19-50585801")

Error: El máximo de datos permitidos fue superado o no puede encontrarse más publicaciones. Se obtuvieron 4000 publicaciones.


In [64]:
print(final_df.shape)
final_df.head()

(4000, 46)


,title,thumbnail_id,category_id,domain_id,currency_id,order_backend,price,available_quantity,tags,seller__id,...,location__state__id,location__state__name,location__latitude,location__longitude,seller_contact__contact,seller_contact__other_info,seller_contact__webpage,seller__home_image_url,days_remaining,years_active
0,2 Ambientes · Suite · Balcón Al Norte · Pileta...,780348-MLA53833280965_022023,MLA401686,MLA-INDIVIDUAL_APARTMENTS_FOR_SALE,USD,1,98900,1,[good_quality_thumbnail],1091275645,...,TUxBUENBUGw3M2E1,Capital Federal,-34.554519,-58.491599,Emiliano Escola Servicios Inmobiliarios · CUCI...,,,NaN,358,0
1,Terreno En Venta De 290m2 Ubicado En Huerta Gr...,723691-MLA53863099879_022023,MLA401687,MLA-INDIVIDUAL_LANDS_FOR_SALE,USD,2,3500,1,[good_quality_thumbnail],322447277,...,TUxBUENPUmFkZGIw,Córdoba,-31.072561,-64.479284,Gustavo,Datos de contacto,,https://http2.mlstatic.com/storage/vis-account...,359,4
2,Casa Interno #200-300 - Pilar Del Este - San R...,921921-MLA53831358637_022023,MLA401685,MLA-INDIVIDUAL_HOUSES_FOR_SALE,USD,3,115000,1,[good_quality_thumbnail],161600541,...,TUxBUEdSQWU4ZDkz,Bs.As. G.B.A. Norte,-34.380938,-58.864169,Federico Cabral,,,NaN,358,8
3,Departamento - Saavedra - 3 Ambientes - Terraz...,727974-MLA53868531819_022023,MLA401686,MLA-INDIVIDUAL_APARTMENTS_FOR_SALE,USD,4,149000,1,[good_quality_thumbnail],1091275645,...,TUxBUENBUGw3M2E1,Capital Federal,-34.558742,-58.486728,Emiliano Escola Servicios Inmobiliarios · CUCI...,,,NaN,359,0
4,Edificio - Colegiales,831521-MLA53856757494_022023,MLA401806,MLA-DEVELOPMENT_APARTMENTS_FOR_SALE,USD,5,76100,1,[good_quality_thumbnail],163427625,...,TUxBUENBUGw3M2E1,Capital Federal,-34.569538,-58.446677,Castex Capital,asdasd223,www.castex.com.ar,NaN,18244,8


In [55]:
final_df.columns

Index(['id', 'title', 'thumbnail_id', 'listing_type_id', 'permalink',
       'category_id', 'domain_id', 'order_backend', 'price', 'sold_quantity',
       'available_quantity', 'tags', 'shipping__logistic_type',
       'shipping__mode', 'shipping__store_pick_up', 'shipping__free_shipping',
       'shipping__tags', 'seller__id', 'seller__nickname', 'seller__tags',
       'seller__seller_reputation__level_id',
       'seller__seller_reputation__power_seller_status',
       'seller__seller_reputation__transactions__canceled',
       'seller__seller_reputation__transactions__completed',
       'seller__seller_reputation__transactions__ratings__negative',
       'seller__seller_reputation__transactions__ratings__neutral',
       'seller__seller_reputation__transactions__ratings__positive',
       'seller__seller_reputation__transactions__total',
       'seller__seller_reputation__metrics__sales__period',
       'seller__seller_reputation__metrics__sales__completed',
       'seller__seller_r

Unimos todos los csv en un solo DataFrame para importarlo para el Modelo

In [2]:
import glob
import os
import pandas as pd

path = r'C:\Users\Juan\OneDrive\Desktop\TPFinalDataScience MELI\Data' # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)

concatenated_df

#li = []

#for filename in all_files:
    #df = pd.read_csv(filename, index_col=None, header=0)
    #li.append(df)

#frame = pd.concat(li, axis=0, ignore_index=True)

#frame.head()

,id,title,thumbnail_id,listing_type_id,permalink,category_id,domain_id,order_backend,price,sold_quantity,...,seller__seller_reputation__metrics__delayed_handling_time__value,seller__seller_reputation__metrics__cancellations__period,seller__seller_reputation__metrics__cancellations__rate,seller__seller_reputation__metrics__cancellations__value,installments__quantity,installments__amount,installments__rate,days_remaining,years_active,variation_filters
0,MLA926762551,Cámara Instantánea Fujifilm Instax Mini 11 Ice...,788648-MLA42359217261_062020,gold_special,https://www.mercadolibre.com.ar/camara-instant...,MLA1040,MLA-ANALOG_CAMERAS,1,54999.00,50,...,0,60 days,0.0000,2,6.0,13598.50,48.35,7242,15,NaN
1,MLA1205153058,Tarjeta De Memoria Sandisk Sdsqxcy-064g-gn6ma ...,665416-MLA40351780042_012020,gold_special,https://www.mercadolibre.com.ar/tarjeta-de-mem...,MLA9467,MLA-MEMORY_CARDS,2,7300.00,250,...,0,60 days,0.0000,0,6.0,1804.93,48.35,7175,0,NaN
2,MLA1124760728,Sony Cyber-shot H300 Dsc-h300 Compacta Avanza...,852025-MLA49434476863_032022,gold_special,https://www.mercadolibre.com.ar/sony-cyber-sho...,MLA1042,MLA-DIGITAL_CAMERAS,3,99999.00,50,...,190,60 days,0.0000,0,6.0,24724.75,48.35,6936,8,NaN
3,MLA1259217110,Cámara De Video Noblex Acn1 4k Negra,863981-MLA42918720182_072020,gold_special,https://www.mercadolibre.com.ar/camara-de-vide...,MLA392132,MLA-VIDEO_CAMERAS,4,19999.00,150,...,617,60 days,0.0085,242,6.0,4944.75,48.35,7211,5,NaN
4,MLA673528318,Ocular Dk-25 Para Nikon D5500 D5300 D5200 D330...,773832-MLA31016831796_062019,gold_special,https://articulo.mercadolibre.com.ar/MLA-67352...,MLA70304,MLA-PHOTOGRAPHY_ACCESSORIES_AND_SPARE_PARTS,5,899.00,200,...,0,60 days,0.0000,0,6.0,222.28,48.35,5244,20,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19986,MLA806671534,Hidrolavadora Philco 105 Bar 1400w Con Espumero,939546-MLA45226866923_032021,gold_pro,https://articulo.mercadolibre.com.ar/MLA-80667...,MLA30840,MLA-ELECTRIC_PRESSURE_WASHERS,46,24363.00,250,...,2020,60 days,0.0022,58,6.0,4060.50,0.00,6005,5,NaN
19987,MLA1123490998,Bordeadora Nober 212 Max Color Naranja 220v,661228-MLA46192795432_052021,gold_pro,https://www.mercadolibre.com.ar/bordeadora-nob...,MLA411878,MLA-ELECTRIC_BRUSH_CUTTERS_AND_STRING_TRIMMERS,47,36814.00,5,...,8,60 days,0.0000,0,6.0,6135.67,0.00,6927,5,NaN
19988,MLA908766072,Minitorno Dremel 4000 220v 175 W Eléctrico + ...,705010-MLA44922506957_022021,gold_special,https://articulo.mercadolibre.com.ar/MLA-90876...,MLA10072,MLA-MICRO_ROTARY_TOOLS,48,31830.00,500,...,0,60 days,0.0000,0,6.0,7869.97,48.35,7279,2,NaN
19989,MLA766557238,Pinza Alicate Multiherramientas Stanley 16 Fun...,860969-MLA40009684434_122019,gold_special,https://articulo.mercadolibre.com.ar/MLA-76655...,MLA412520,MLA-TOOL_AND_CONSTRUCTION_SUPPLIES,49,3440.00,500,...,2020,60 days,0.0022,58,6.0,850.54,48.35,6970,5,NaN


In [3]:
concatenated_df.columns

Index(['id', 'title', 'thumbnail_id', 'listing_type_id', 'permalink',
       'category_id', 'domain_id', 'order_backend', 'price', 'sold_quantity',
       'available_quantity', 'tags', 'shipping__logistic_type',
       'shipping__mode', 'shipping__store_pick_up', 'shipping__free_shipping',
       'shipping__tags', 'seller__id', 'seller__nickname',
       'seller__car_dealer', 'seller__tags', 'seller__car_dealer_logo',
       'seller__seller_reputation__level_id',
       'seller__seller_reputation__power_seller_status',
       'seller__seller_reputation__transactions__canceled',
       'seller__seller_reputation__transactions__completed',
       'seller__seller_reputation__transactions__ratings__negative',
       'seller__seller_reputation__transactions__ratings__neutral',
       'seller__seller_reputation__transactions__ratings__positive',
       'seller__seller_reputation__transactions__total',
       'seller__seller_reputation__metrics__sales__period',
       'seller__seller_reputati

In [4]:
concatenated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19991 entries, 0 to 19990
Data columns (total 47 columns):
 #   Column                                                             Non-Null Count  Dtype  
---  ------                                                             --------------  -----  
 0   id                                                                 19991 non-null  object 
 1   title                                                              19991 non-null  object 
 2   thumbnail_id                                                       19991 non-null  object 
 3   listing_type_id                                                    19991 non-null  object 
 4   permalink                                                          19991 non-null  object 
 5   category_id                                                        19991 non-null  object 
 6   domain_id                                                          19991 non-null  object 
 7   order_backend         

## Detalle, hay ciertas categeorias que tienen otras columnas, quizas ver si categoria inmuebles aparte, asi como tambien autos.

Convertimos el dataFrame final concatenado en un solo archivo CSV

In [ ]:
concatenated_df.to_csv(f'./Data/dfUnido_meli.csv', index=False)
